# News To Stock Analyser 데이터준비

뉴스기사를 전달하면, 긍/부정분석 뿐아니라, 특정주식에 대한 긍/부정평가 처리 RAG 구현

In [2]:
%pip install -Uq datasets langchain-openai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")

## 데이터준비
https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko

In [4]:
from datasets import load_dataset  # HuggingFace 데이터셋 로더

dataset = load_dataset('daekeun-ml/naver-news-summarization-ko')  # 데이터셋 다운로드 및 로드
dataset

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--daekeun-ml--naver-news-summarization-ko. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 2740/2740 [00:00<00:00, 19363.89 examples/s]


DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [5]:
# 필터링 : train 데이터셋에서 category가 economy만 남김
economy_dataset = dataset['train'].filter(lambda row: row['category'] == 'economy')
print(len(economy_dataset))

Filter: 100%|██████████| 22194/22194 [00:00<00:00, 53936.73 examples/s]

17088


In [6]:
economy_dataset[10]  # economy_dataset에서 11번째 샘플 반환

{'date': '2022-07-01 06:54:01',
 'category': 'economy',
 'press': 'SBS Biz ',
 'title': '글로벌 비즈 가트너 올해 전세계 스마트폰 판매량 7% 감소 전망',
 'document': '경제와이드 모닝벨 글로벌 비즈 임선우 외신캐스터 글로벌 비즈입니다. ◇ 올해 스마트폰 판매 감소 올해 전세계 스마트폰 판매량이 크게 줄어들 것이란 전망이 나왔습니다. 시장조사업체 가트너는 글로벌 스마트폰 판매가 7% 하락할 것으로 내다봤는데요. 경제 전반에 걸친 침체 우려와 중국의 봉쇄조치 여파 그리고 인플레이션으로 소비자들이 지갑을 열기 주저하면서 수요가 줄어들 것 이라고 설명했습니다. 그러면서 올해 전체 출하량은 14억6천만대 수준에 그칠 것으로 예측했는데요. 종전 전망치인 16억대에서 대폭 낮춰 잡았습니다. 특히 세계 최대 스마트폰 시장인 중국에서 판매량은 18%가 감소할 것으로 전망했는데요. 가트너는 이같은 수요 부진으로 애플을 비롯한 스마트폰 제조사부터 엔비디아 TSMC 같은 반도체 업체까지 압력이 가해질 것이라고 진단했습니다. ◇ EU 가상자산 돈세탁 막는다 유럽연합이 가상자산을 이용한 돈세탁을 막기위해 관련 기업을 규제하는 방안에 잠정 합의했습니다. 잠정안에는 가상자산 업체가 당국에 모든 디지털자산 거래에 대한 신원 확인 정보를 제공하도록 하는 내용이 담겼는데요. 이에 따라 업체들은 관련 개인정보를 확보해야하고 당국이 이를 요구할 경우 제출해야 합니다. 또 거래액이 1천 유로 우리돈 130만 원을 넘길 경우 비인증 거래소가 관리하는 가상자산 지갑도 똑같은 규칙이 적용되는데요. 여기에 더해 송금 규제를 활용해 거래를 상시 추적하고 불법성이 의심되는 거래를 막을 수 있도록 할 방침입니다. 이와 관련해 미국 최대 가상자산 거래소 코인베이스 등 관련 기업 40여 곳은 개인정보 침해 가능성을 언급하며 줄곧 반대 입장을 밝혀왔는데요. 하지만 최근 가상자산 관련 범죄 사례가 급증하고 있는 만큼 규제 움직임이 힘을

In [7]:
import pandas as pd

df = economy_dataset.to_pandas()  # Dataset을 DataFrame 형식으로 변환
df.head()

,date,category,press,title,document,link,summary
0,2022-07-03 17:14:37,economy,YTN,추경호 중기 수출지원 총력 무역금융 40조 확대,앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 ...,https://n.news.naver.com/mnews/article/052/000...,"올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, ..."
1,2022-07-04 08:07:12,economy,아시아경제,해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다,문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준...,https://n.news.naver.com/mnews/article/277/000...,인터엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타...
2,2022-07-01 08:51:12,economy,뉴시스,에디슨이노 이승훈 제이스페이스 대표 사내이사 선임,기사내용 요약 우주발사체 사업 본격화 서울 뉴시스 김경택 기자 에디슨이노가 우주발사...,https://n.news.naver.com/mnews/article/003/001...,에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이...
3,2022-07-01 16:11:01,economy,머니투데이,SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차,SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사...,https://n.news.naver.com/mnews/article/008/000...,SK바이오사이언스가 글로벌 사업의 고도화를 위해 기존 해외사업개발실을 백신사업뿐만 ...
4,2022-07-01 21:48:04,economy,경향신문,금융당국 “증시 변동성 완화 조치”,4일부터 석달간 증권사 신용융자담보비율 유지의무 면제 금융당국이 코스피지수가 장중 ...,https://n.news.naver.com/mnews/article/032/000...,1일 1일 금융위원회는 증권 유관기관과 금융시장합동점검회의를 열고 코스피지수가 장중...


## sLLM 답변데이터 생성
llm을 이용해서 sLLM이 답변했으면 하는 내용을 생성해낸다. 이때 답변을 품질이 중요하므로, 되도록 상위모델을 사용하는 것이 좋다.

In [12]:
# 금융뉴스 분석용 구조화 출력 스키마(Pydantic) + 프롬프트 템플릿 구성
from pydantic import BaseModel, Field  # 구조화 출력 스키마 정의
from typing import List, Optional      # 타입 힌트
from langchain_core.prompts import ChatPromptTemplate  # 채팅 프롬프트 템플릿

class StockAnalysis(BaseModel):
    stock_related: bool = Field(description='뉴스와 주식 종목간의 연관성 여부')
    summary: str = Field(description='뉴스 요약')

    # 긍정적 영향 관련 변수 (default_factory=list : 인스턴스가 생성될 때마다 list() 호출해서 새 리스트를 만들어준다.)
    positive_stocks: List[str] = Field(description='긍정적 영향이 예상되는 주식 종목명 목록', default_factory=list)
    positive_keywords: List[str] = Field(description='긍정적 영향의 근거가 되는 키워드 목록', default_factory=list)
    positive_reasons: List[str] = Field(description='긍정적 영향이 예상되는 이유')
    
    # 부정적 영향 관련 변수
    negative_stocks: List[str] = Field(description='부정적 영향이 예상되는 주식 종목명 목록', default_factory=list)
    negative_keywords: List[str] = Field(description='부정적 영향의 근거가 되는 키워드 목록', default_factory=list)
    negative_reasons: List[str] = Field(description='부정적 영향이 예상되는 이유')

system_prompt = '''  # 모델 역할/출력 규칙을 고정하는 시스템 프롬프트
당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고,
특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.

다음 출력지시사항을 지켜주세요.
1. 뉴스와 종목간의 연관성을 발견할 수 없다면:
    - stock_related를 False로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
2. 뉴스와 종목간의 연관성을 발견했다면:
    - stock_related를 True로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.
    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.
    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.
'''

user_prompt = '''  # 분석 대상 뉴스 본문을 전달하는 사용자 프롬프트
다음 뉴스기사의 내용에 대해 심층적인 분석을 수행해주세요.

[news]
{news}
'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', user_prompt)
])

news = df['document'][1]
prompt.invoke({'news': news})

ChatPromptValue(messages=[SystemMessage(content="  # 모델 역할/출력 규칙을 고정하는 시스템 프롬프트\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고,\n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n다음 출력지시사항을 지켜주세요.\n1. 뉴스와 종목간의 연관성을 발견할 수 없다면:\n    - stock_related를 False로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n2. 뉴스와 종목간의 연관성을 발견했다면:\n    - stock_related를 True로 작성하세요.\n    - summary에 뉴스의 요약을 작성하세요.\n    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.\n    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.\n    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='  # 분석 대상 뉴스 본문을 전달하는 사용자 프롬프트\n다음 뉴스기사의 내용에 대해 심층적인 분석을 수행해주세요.\n\n[news]\n문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준비 7 8월 2만5000원 추가 시 와인 5종 및 생맥주 무제한 제공 인터컨티넨탈 서울 코엑스 브래서리 쿨 섬머 페스타 . 인터컨티넨탈 서울 코엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타 를 진행한다고 4일 밝혔다. 미국식 해산물 요리인 시푸드 보일 을 대표 메뉴로 선보이며 소믈리에 추천 와인 5종과 생맥주를 무제한 제공하는 주류 프로모션

In [13]:
from langchain.chat_models import init_chat_model

llm = init_chat_model('openai:gpt-4.1')

chain = prompt | llm.with_structured_output(StockAnalysis)  # 프롬프트 -> LLM -> StockAnalysis 구조화 출력

def analyze_news(news):
    return chain.invoke({'news': news})

analyze_news(news)

StockAnalysis(stock_related=True, summary="인터컨티넨탈 서울 코엑스의 1층 뷔페 레스토랑 브래서리가 7~8월 동안 '쿨 섬머 페스타'를 개최하여 미국식 해산물 요리(시푸드 보일) 및 다양한 해산물 메뉴, 그리고 와인·생맥주 무제한 프로모션을 제공한다. 추가 요금을 내면 다양한 주류도 즐길 수 있고, 10만원 이상 와인 구매 시 선물도 증정된다. 가격은 기존과 동일하며 할인 혜택도 제공된다.", positive_stocks=['호텔신라', '신세계', 'AJ네트웍스'], positive_keywords=['해산물 음식 프로모션', '호텔 뷔페', '고객 유치', '여름 성수기', '주류 판매 증가'], positive_reasons=['여름 휴가 및 성수기를 맞아 호텔 레스토랑 프로모션 강화로 방문객 유치 효과 기대', '호텔업 및 외식업 관련 기업의 매출 증가 가능성', '고가의 와인 및 주류 상품 패키지 운영으로 객단가 상승 효과 기대'], negative_stocks=[], negative_keywords=[], negative_reasons=[])

In [14]:
news = df['document'][100]
display(news)
print()

analyze_news(news)

'해수부 5일 개정 공유수면 관리 및 매립에 관한 법률 시행 헤럴드경제 홍태화 기자 앞으로 공유수면관리청이 어업·환경 등에 영향을 미칠 것으로 예상되는 공유수면 점용·사용 허가를 할 때 미리 어업인 등 이해관계자들의 의견을 들어야 한다. 해양수산부는 5일 이같은 내용이 담긴 개정 공유수면 관리 및 매립에 관한 법률과 같은 법 시행령·시행규칙이 이날부터 시행된다고 밝혔다. 바다·바닷가·하천 등 공유수면은 공유재이기 때문에 이를 점용·사용하기 위해서는 별도의 허가를 받아야 한다. 최근 해상풍력 발전시설 해변을 이용한 관광시설 등 대규모 시설이 공유수면을 장기적으로 점용·사용하는 경우가 늘어났지만 이해 관계자의 의견을 사전에 수렴할 수 없는 문제가 있었다. 이에 공유수면 점용·사용으로 인한 사회적 갈등이 증가했다. 이러한 문제를 해결하기 위해 해수부는 지난 1월 공유수면 점용·사용 허가를 할 때 이해관계자의 의견을 듣도록 공유수면 관리 및 매립에 관한 법률을 개정했다. 법 개정에 따라 공유수면관리청이 해양환경·수산자원·자연경관 보호 등에 영향을 끼칠 수 있는 공유수면 점용·사용 신청을 받은 경우 이를 관보 공보 와 인터넷 홈페이지에 공고해야 한다. 또 점용·사용 허가를 했을 때 피해를 볼 것으로 예상되는 어업인에 대한 의견 조사도 별도로 진행해야 한다. 황준성 해수부 해양공간정책과장은 공유수면 점용·사용으로 인한 이해 관계자의 피해를 방지하려는 법령 개정의 취지를 달성할 수 있도록 각 공유수면관리청과 협력해 관련 제도의 차질 없는 운영을 지원하겠다 고 말했다.'

StockAnalysis(stock_related=True, summary='해양수산부가 시행하는 개정 공유수면 관리 및 매립법에 따라 앞으로 해상풍력, 관광시설 등 공유수면 점용·사용 허가 시 어업인 등 이해관계자의 의견을 사전에 수렴해야 한다. 해양환경, 수산자원 등을 고려한 사전 공고 및 피해 예상 어업인 의견조사 절차가 신설된다. 이는 공유수면을 이용한 대규모 개발사업의 추진 절차가 강화되는 것을 의미한다.', positive_stocks=[], positive_keywords=[], positive_reasons=[], negative_stocks=['씨에스윈드', '씨에스베어링', '대한해운', '현대에너지솔루션', 'SK에코플랜트', '두산에너빌리티'], negative_keywords=['공유수면', '해상풍력', '허가절차 강화', '이해관계자 의견수렴', '갈등조정', '개발지연'], negative_reasons=['공유수면을 이용한 해상풍력발전 등 대규모 인프라 개발 기업은 허가 절차가 강화되며 이해관계자 협의 과정이 추가되어 사업 추진 지연 및 비용 증가 우려가 있음', '해상풍력, 해양관광시설 등 관련 사업자들은 지역 어민 등 이해관계자와의 갈등 증폭 및 행정절차 강화로 인한 불확실성 상승 가능성'])

In [15]:
df = df[:1000]  # 상위 1000개 샘플링
df['content'] = df['title'] + '\n' + df['document']  # 제목 + 본문으로 content 컬럼 생성

pd.set_option('display.max_colwidth', None)  # 컬럼 내용이 잘리지 않도록 설정
df['content'].head()

0                                                                                                                                                                      추경호 중기 수출지원 총력 무역금융 40조 확대\n앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 

In [16]:
# llm 질의 & 분석 데이터 1000개 생성
from tqdm.auto import tqdm    # 프로그레스바

results = []  # 분석 결과 저장 리스트

for content in tqdm(df['content']):
    result = analyze_news(content)  # 분석 진행(StockAnalysis 반환)
    results.append(result)          # 리스트에 추가

df['result'] = results  # 결과 리스트를 DF result 컬럼으로 추가
df.head()

 17%|█▋        | 172/1000 [19:38<1:34:32,  6.85s/it]


KeyboardInterrupt: 

In [ ]:
# json 변환
# - pydantic.BaseModel.model_dump() -> dict
# - pydantic.BaseModel.model_dump_json() -> json_str

# Pydantic 객체를 JSON 문자열로 변환하는 함수
def parse_to_json(obj):
    return obj.model_dump_json()  # StockAnalysis 객체 -> JSON 문자열로 파싱

df['result_json'] = df['result'].apply(parse_to_json)  # result_json 컬럼은 JSON 문자열로 파싱한 결과
df.head()

In [ ]:
df = df.dropna(subset=['result_json'])  # 결측 행 제거
df = df.reset_index(drop=True)          # 인덱스 0부터 재구성
df.head()

## 학습용 데이터셋 변환
- system
- user(human)
- assistant(ai)

In [ ]:
# 파이튜닝용 (system/user/assistant) 컬럼 구성 및 출력
df['system'] = system_prompt

# 키로 들어온 컬럼명을 값으로 들어온 컬럼명으로 변경
df = df.rename(columns={
    'content': 'user',
    'result_json': 'assistant'
})

df[['system', 'user', 'assistant']]

In [ ]:
# DataFrame을 JSON 파일로 저장
df[['system', 'user', 'assistant']].to_json(
    'train_json',         # 저장 파일명(경로)
    orient = 'records',   # 리스트 형태로 저장 
    force_ascii = False,  # 한글 깨짐 방지
    indent = 4            # 들여쓰기 4칸
)

In [ ]:
import os
from datasets import Dataset

dataset = Dataset.from_pandas(df[['system', 'user', 'assistant']])  # DataFrame -> Dataset 변환

# HuggingFace Hub에 업로드
dataset.push_to_hub(
    'capybaraOh/naver-economy-news2stock',  # 업로드할 repo 이름
    token = os.environ['HF_TOKEN']          # 인증 토큰
)

---
시간상 데이터셋 생성 -> 가공 -> 업로드까지는 아래 코드로 대체

In [24]:
from datasets import load_dataset

ds = load_dataset("capybaraOh/naver-economy-news2stock")
ds

DatasetDict({
    train: Dataset({
        features: ['system', 'user', 'assistant'],
        num_rows: 1000
    })
})

In [25]:
ds.push_to_hub(
    'capybaraOh/naver-economy-news2stock',
    token = os.environ['HF_TOKEN']
)

,system,user,assistant
0,"\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, \n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n다음 출력지시사항을 지켜주세요.\n1. 뉴스와 종목간의 연관성을 발견할 수 없다면:\n - stock_related를 False로 작성하세요.\n - summary에 뉴스의 요약을 작성하세요.\n2. 뉴스와 종목간의 연관성을 발견했다면:\n - stock_related를 True로 작성하세요.\n - summary에 뉴스의 요약을 작성하세요.\n - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.\n - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.\n - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.\n",추경호 중기 수출지원 총력 무역금융 40조 확대\n앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.,"{""stock_related"":true,""summary"":""정부가 중소기업, 중견기업의 수출 확대를 지원하기 위해 무역금융을 40조원 이상 추가 확대(총 301조원)하고, 물류비 지원·임시선박 투입 등 실질 지원책을 추진한다. 또한 반도체 등 첨단산업 육성, 에너지 효율화 방안 등 종합적 무역수지 개선책을 병행할 예정이다."",""positive_stocks"":[""한국항공우주"",""CJ대한통운"",""현대글로비스"",""삼성전자"",""SK하이닉스""],""positive_keywords"":[""무역금융 확대"",""수출 지원"",""물류비 지원"",""임시선박 투입"",""반도체 육성"",""수출 증가세""],""positive_reasons"":""수출기업·물류기업·중소/중견 제조업체의 자금난 및 물류난 해소로 직접적 매출 확대가 기대되며, 특히 반도체 등 첨단산업은 정부의 집중 지원으로 실적 개선 및 성장 모멘텀 확보가 예상됨."",""negative_stocks"":[],""negative_keywords"":[],""negative_reasons"":""""}"
1,"\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, \n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n다음 출력지시사항을 지켜주세요.\n1. 뉴스와 종목간의 연관성을 발견할 수 없다면:\n - stock_related를 False로 작성하세요.\n - summary에 뉴스의 요약을 작성하세요.\n2. 뉴스와 종목간의 연관성을 발견했다면:\n - stock_related를 True로 작성하세요.\n - summary에 뉴스의 요약을 작성하세요.\n - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.\n - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.\n - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.\n",해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다\n문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준비 7 8월 2만5000원 추가 시 와인 5종 및 생맥주 무제한 제공 인터컨티넨탈 서울 코엑스 브래서리 쿨 섬머 페스타 . 인터컨티넨탈 서울 코엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타 를 진행한다고 4일 밝혔다. 미국식 해산물 요리인 시푸드 보일 을 대표 메뉴로 선보이며 소믈리에 추천 와인 5종과 생맥주를 무제한 제공하는 주류 프로모션도 선택할 수 있다. 시푸드 보일 이 대표 메뉴로 준비되고 라이브 스테이션에서 셰프가 직접 원하는 메뉴를 먹기 좋게 잘라 제공한다. 시푸드 보일은 문어와 랍스터 대게 갑오징어 새우 소라 관자 낙지 등 해산물을 쪄낸 뒤 셰프의 비법 시즈닝으로 이국적인 감칠맛을 더한 메뉴다. 프로모션 기간에는 해물전 가리비 불도장 장어 데마끼 로제 해물 뇨끼 등 한식 중식 일식 양식 등 세계 각국의 해산물 메뉴도 즐길 수 있다. 소믈리에 추천 와인 5종과 생맥주를 무제한으로 제공하는 옵션도 선택할 수 있다. 제공되는 와인은 레드와 화이트 와인 각 2종 스파클링 와인 1종으로 취향에 따라 다양하게 즐길 수 있다. 해당 기간 동안 입구 와인셀렉션 코너에서 10만원 이상 와인 구매 시 호텔에서 제작한 주트백도 선물로 증정한다. 이용 가격은 이전과 동일하며 네이버 예약 시 10% 할인 혜택도 제공한다. 주류 무제한 혜택은 2만5000원 추가 시 이용할 수 있다.,"{""stock_related"":true,""summary"":""인터컨티넨탈 서울 코엑스의 뷔페 레스토랑 브래서리가 여름 한정 '쿨 섬머 페스타'를 6월부터 8월 말까지 진행한다. 이 행사는 해산물(문어, 랍스터, 대게 등)을 활용한 미국식 해물찜 '시푸드 보일' 및 다양한 세계 각국 해산물 요리, 그리고 2만5000원 추가 시 와인 5종과 생맥주 무제한 제공 등의 주류 프로모션을 포함한다. 프로모션 기간 내 와인 구매 시 사은품 증정과 예약 할인 혜택도 있다."",""positive_stocks"":[""호텔신라"",""신세계"",""현대백화점"",""롯데칠성음료""],""positive_keywords"":[""호텔"",""뷔페"",""해산물 소비 증가"",""주류 소비 증가"",""프리미엄 외식"",""여름 한정 프로모션""],""positive_reasons"":""여름철 프리미엄 호텔 뷔페의 해산물 및 주류 무제한 프로모션은 호텔 체인(호텔신라, 신세계, 현대백화점 등) 및 프리미엄 식음료 공급업체(롯데칠성음료 등)에 긍정적입니다. 여름 성수기 고객 유입 증가와 고급 식자재 소비 확대, 와인 및 주류 수요 증가로 외식 및 F&B 관련 매출 증대가 기대됩니다."",""negative_stocks"":[],""negative_keywords"":[],""negative_reasons"":""""}"
2,"\n당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고, \n특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.\n\n